In [31]:

import pandas as pd
import numpy as np

In [32]:
data=pd.read_csv(r".\data\gemstone.csv")
data.drop(labels=["id"],axis=1,inplace=True)
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619.0
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387.0
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772.0
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666.0
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453.0


In [33]:
X=data.drop(labels=["price"],axis=1)
y=data[["price"]]

In [4]:

categorical_cols=X.select_dtypes(include='object').columns

numerical_cols=X.select_dtypes(exclude='object').columns

In [5]:
X.select_dtypes(include='object')

,cut,color,clarity
0,Premium,F,VS2
1,Very Good,J,SI2
2,Ideal,G,VS1
3,Ideal,G,VS1
4,Premium,G,VS2
...,...,...,...
193568,Ideal,D,VVS2
193569,Premium,G,VVS2
193570,Very Good,F,SI1
193571,Very Good,D,SI1


In [8]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:

from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [10]:
num_pipeline=Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())
    ]
)
    

In [16]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [11]:
cat_pipeline=Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[ cut_categories
                                                         , color_categories
                                                         , clarity_categories
                                                    ]))
    ]
)

In [15]:
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good',
                                             'Premium', 'Ideal'],
                                            ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                                            ['I1', 'SI2', 'SI1', 'VS2', 'VS1',
                                             'VVS2', 'VVS1', 'IF']]))])

In [12]:
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline', num_pipeline, numerical_cols),
        ('cat_pipeline', cat_pipeline, categorical_cols)
    ]
)

In [14]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']]))]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [17]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [18]:

preprocessor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  4.        ,
         5.        ,  5.        ],
       [ 0.2351953 ,  1.83363716, -0.12153081, ...,  1.        ,
         1.        ,  2.        ],
       [ 0.49461699,  0.81585507,  0.39980029, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ...,  1.        ,
         3.        ,  2.        ],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  4.        ,
         3.        ,  2.        ]])

In [19]:

preprocessor.transform(X_test)

array([[-0.56468825, -0.94213218, -0.6428619 , ...,  3.        ,
         1.        ,  3.        ],
       [-0.17555571,  1.00090636, -0.12153081, ...,  2.        ,
         4.        ,  2.        ],
       [-1.06191316,  0.2607012 , -0.12153081, ...,  4.        ,
         4.        ,  7.        ],
       ...,
       [-0.19717419, -3.34779894,  1.44246248, ...,  3.        ,
         6.        ,  3.        ],
       [-0.82410994, -0.20192702, -0.12153081, ...,  4.        ,
         3.        ,  2.        ],
       [ 2.61322747, -0.75708089,  1.44246248, ...,  3.        ,
         6.        ,  3.        ]])

In [ ]:
preprocessor.get_feature_names_out()

In [20]:

X_train=pd.DataFrame(preprocessor.fit_transform(X_train)
                     , columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test)
                    , columns=preprocessor.get_feature_names_out())

In [21]:

"""linear regression
ridge regression
lasso regression
elastic net"""

'linear regression\nridge regression\nlasso regression\nelastic net'

In [22]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [23]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [24]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [25]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [28]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    # print(model)

In [29]:
models

{'LinearRegression': LinearRegression(),
 'Lasso': Lasso(),
 'Ridge': Ridge(),
 'Elasticnet': ElasticNet()}

In [30]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2 score", r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.9047094344004
MAE: 674.0255115796833
R2 score 93.68908248567512


Lasso
Model Training Performance
RMSE: 1013.8723151049944
MAE: 675.2036880701619
R2 score 93.68948574778251


Ridge
Model Training Performance
RMSE: 1013.9058997761061
MAE: 674.056513229593
R2 score 93.68906766741328


Elasticnet
Model Training Performance
RMSE: 1513.9140350229763
MAE: 1050.7468664314322
R2 score 85.92978759337907


